# Methods

## Hemodynamic Response Function (HRF)

Generally speaking, cognitive processing is associated with increases in neuronal firing rates. The increased neural activity leads to increased metabolic requirements for the neurons. The onset of neural activity leads to a systematic series of physiological changes in the local network of blood vessels that include changes in the cerebral blood volume per unit of brain tissue (CBV), changes in the rate of cerebral blood flow (CBF), and changes in the concentration of oxyhaemoglobin and deoxyhaemoglobin.

The most common functional imaging signal is the Blood Oxygenation Level Dependent signal (BOLD), which primarily corresponds to the concentration of deoxyhaemoglobin.

For the purposes of estimating the BOLD signal in an experimental paradigm, SPM makes use of a canonical haemodynamic response function (HRF). This function is assumed to be the response of the system (as reflected by the MR signal) to a brief, intense period of neural stimulation. The canonical HRF in SPM is defined as the difference of two Gamma function. The implementation, including the shape parameters, was ported from the SPM matlab implementation into julia. 

$h(t) = A (\frac{t^{\alpha_1 - 1} \beta^{\alpha_1} e^{-\beta_1 t}}{\Gamma{\alpha_1}} - c \frac{t^{\alpha_2 -1} \beta^{\alpha_2} e^{-\beta_2 t}}{\Gamma (\alpha_2) } ) $

The Repetition Time or Repeat Time (RT) parameter is passed to the hrf function. It is the temporal resolution at which the fmri timeseries was acquired. In the following I will refer to a $hrf$ function with a given RT as $hrf_{RT}$ for clarity. $hrf$ functions with varying Repetition Times are shown below.

![alt text](hrf_comp.png "Title")

## Convolution

Definition Convolution: Convolution is a operation on two functions $f$ and $g$ written as $f \ast g$. It is defined as
    $(f \ast g)(t) := \int_{-\infty}^{\infty} f(t - \tau) g(\tau) d\tau$

Definition discrete convolution: For functions $f,g$ definted on the set $\mathbb{Z}$ of integers, the discrete convolution of $f$ and $g$ is defined as 
    $(f \ast g)[n] = \sum_{m=-\infty}^{\infty}f[m] g[n-m]$

Convolutional Theorem: Consider two functions $f(x)$ and $g(x)$ with Fourier transforms $F(k)=\mathscr{F} \{f\}(k)$ and $G(k)=\mathscr{F} \{g\}(k)$ where $\mathscr{F}$ denotes the Fourier transform operator. Define the convolution of $f$ and $g$ as $r(x) := (f \ast g ) (x)$

The convolutional theorem states that $R(k) := \mathscr{F} \{ r \} (k) = F(k) G(k)$ 

Applying the inverse Fourier transform $\mathscr{F}^{-1}$ produces the corollary: $r(x) = \mathscr{F}^{-1} \{ F \cdot G \}(x)$ where $\cdot$ denotes point-wise multiplication

When $g$ has finite support in the set $\{ -M, -M+1, ..., M-1, M \}$ (representing, for instance, a finite impulse response), a finite summation may be used: 
    $(f \ast g )[n] = \sum_{m=-M}^{M} f[n-m]g[m]$ 

![alt text](boxcar_hrf_conv.png "Title")

## Deconvolution

I followed the deconvolution approach presented in https://www.sciencedirect.com/science/article/pii/S1053811921008648 for fmri time series

Deconvolution is the inverse operation to convolution. Given the functions $g$ and $h$, the objective of deconvolution is to find $f$ given the convolution equation 
    $f \ast g = h$

Usually, $h$ is a recorded signal and $f$ is the signal we wish to recover but has been convolved by a filter or inpulse response function $g$ when measuring/recording the signal. In (physical) measurements, the situation is usually 
    $(f \ast g) + \epsilon = h$ where $\epsilon$ is the noise term of the signal.
    
Assuming white noise we can use the Wiener Deconvolution to recover $f$

Wiener Deconvolution:
    Given a system $y(t) = (h \ast x)(t) + n(t)$

where

$x(t)$ the original signal we wish to recover

$h(t)$ the known impulse response

$n(t)$ additive white noise term, independent of $x(t)$

$y(t)$ observed signal
    
    
The goal is to find a function $g(t)$ so that we can estimate $x(t)$

$\hat{x}(t) = (g \ast y)(t)$
    
where $\hat{x}(t)$ is an estimate of $x(t)$ that minimizes the mean square error
    
The Wiener deconvolution filter provides such a $g(t)$. In the frequency domain the filter is written as

$G(k) = \frac{H*(k)S(k)}{\|H(k)\|^2 S(k) + N(k)}$

where:

$G(k)$ and $H(k)$ are the Fourier transforms of $g(t)$ and $h(t)$

$S(k) = \mathbb{E} [X(k)]^2$ is the mean power spectral density of the original signal $x(t)$

$N(k) = \mathbb{E} [V(k)]^2$ is the mean power spectral density of the noise $n(t)$

$X(k)$, $Y(k)$ and $V(k)$ are the Fourier transforms of $x(t)$, $y(t)$ and $n(t)$ respectively.

the superscript $^*$ denotes complex conjugation

Using the convolution theorem we can then obtain $\hat{X}(k) = G(k)Y(k)$ and perform an inverse Fourier transform on $\hat{X}(k)$ to obtain $\hat{x}(t)$.

An application of the Wiener Deconvolution to the signal $x(t) = \sin(2t) \cdot \cos(5t) - 1.5 \cos(6t) \cdot sin(2t) + 0.2 \sin(20t)$ convoluted with $hrf_{0.2}$ and added white noise is shown below (under the assumption that the Powerspectra of the original signal and noise is known)

![alt text](wiener_deconv.png "Title")

Due to the padding of the signal edges both the observed signal and the deconvolution are longer than the original signal. When calculating metrics like the mean squared error the edges have to be removed to obtain a useful result.

## Wavelets

### Continious Wavelet transform (CWT)

The fundamental idea behind wavelets is to analyze through the scale. Wavelets are used in wavelet transformation the same way sin and cos are used in the Fourier transformation. In Fourier transformation, a function is transformed to a new basis given by sin and cos functions in frequency space.

The basis functions of the Fourier transform are localized in the frequency domain, each sin/cos has a single frequency, but not in the time domain, where they are periodic and non-vanishing. This means that for periodic, continuous signals the Fourier transform only needs a small amount of coefficients to obtain a good approximation of the signal. In addition, noise limited to certain frequencies can be easily detected and filtered. 

On the other hand, local signals containing discontinuities cannot be approximated easily and one needs a large amount of Fourier coefficients. Furthermore, localized time information is stored in the phase of the Fourier transformation, so modifications to the signals in frequency space e.g. filtering may corrupt transient time information, leading to unwanted side effects.

These problems are addressed by wavelet transformations. Wavelets are, just like sin and cos in the Fourier transformation, used as basis functions
to represent a signal in the frequency domain. The major difference is that wavelets are localized in time and frequency domain. Therefore, wavelets are well-suited to approximate data containing sharp discontinuities. Another advantage of wavelets is that their width in time and frequency can vary, as illustrated in Figure below. This way one has long low frequency basis function for frequency analysis and short basis function to isolate discontinuities. The continuous wavelet transform can be expressed similarly to Fourier transform

$F(a,b) = \int_{\mathbb{R}}f(x)\psi_{(a,b)}^{\ast}(x)dx$

where * is the complex conugate and $\psi$ is a given wavelet function.

![alt text](STFT_and_WT.jpg "Title")

Source: https://commons.wikimedia.org/wiki/File:STFT_and_WT.jpg

Definition: A function $\psi \in L^{2}(\mathbb{R})$ is called a Wavelet if it satifies the following conditions

   1. meanfree $\int_{\mathbb{R}} \psi (t) dt = 0$

   2. normalised $\sqrt{\int_{\mathbb{R}} \psi(t) \psi^{\ast}(t) dt} = 1$

In many cases an additional condition is required:

   3. admissibility $\int_{\mathbb{R}} \frac{\psi(t) \psi^{\ast}(t)}{\|t\|} dt < \infty$

From a so called Mother wavelet that satisfiesthe conditions above furthur Daughter wavelets can be generated using scaling and translation

$\psi_{s, \tau} = \frac{1}{s} \psi(\frac{t-\tau}{s})$

The scale factor $s$ and the translation factor $\tau$ determine the wavelet's width and position. This entire Wavelet family defines an orthogonal basis. By construction the basis elements are self-similar, once we know about the mother function we know everything about the basis.

### Discrete Wavelet Transformation

The discrete wavelet transformation (DWT) of a signal $x[n]$ can be represented as a series of filters. On each level, the signal passes through a low pass filter, resulting in the so called approximation coefficients, and a high pass filter, resulting in the so called detail coefficients.

$y_{\text{high/low}}[n] = (x \ast f)[n] = \sum_{k \in $\mathbb{Z}$} x[k]f[n-k]$

with $f$ being the filter function.  Since approximately half the frequencies have been removed, half the samples can be discarded according to Nyquit’s rule. Using the sub-sampling operator $\downarrow$

$(f \downarrow k)(n) = f(kn)$

we can write the transformation as 

$y_{high} = (x \ast h) \downarrow 2$
$y_{low} = (x \ast g) \downarrow 2$

To further increase the frequency resolution, this decomposition can be recursively applied taking the approximation coefficients of a given level as input for the next. This so called filter bank is shown schematically below. The filters are coefficients {c0, ..., cn} calculated with help of the mother wavelet.

![alt text](DWT_scheme.png "Title")

### Noise Estimation and Denoising

The Wiener Deconvolution needs an estimate of the power spectra of the noise and the original signal. 

The noise was assumed to be gaussian white noise with mean $\mu=0$.

As proposed by Donoho 1995, the standard deviation was estimated by $\hat{\sigma} = MAD/0.6745$ where $MAD$ is the median absolute value of the normalized finescale wavelet coefficients. (https://www.fceia.unr.edu.ar/~jcgomez/wavelets/Donoho_1995.pdf)

To estimate the original signal the noisy signal was denoised using the so called VisuShrink method, a wavelet based denoising technique. The Algorithm can be summerized as follows:

1. Apply the wavelet transformation $W$ to the signal and obtain the empirical wavelet coefficients.

2. Apply thresholding to the wavelet coefficients. I used hard thresholding in this work $\Theta^0_T(x)_i = \{ x_i \text{ if } \|x_i\|>T,  0 \text{ otherwise } \}$

3. Apply the inverse wavelet transformation $W^{\ast}$ to thresholded coefficients to obtain the final estimator of the signal $\tilde{f} = W^*(\tilde x)$

(https://academic.oup.com/biomet/article/81/3/425/256924?login=true)

![alt text](wavelet_denoise.png "Title")

## Observation Model

The observation equation used in identity TF is

$\hat{x}_{t} = B z_{t}$

where $\hat{x}_{t}$ is the estimate of the N-dimension data point $x_t$, $z_{t}$ is the M-dimensional hidden state vector and $B$ is a fixed $N \times M$ matrix $\mathcal{I}$ with entires

$\mathcal{I}_{i,j} = \{1 \text{ if } i=j \land i,j \leq N, 0 \text{ else} \}$

In the case of $N=M$ this simplifies to $B=\mathbb{1}$ the identity matrix


The implication of using $\mathcal{I}$ is splitting $z$ into an observation and a hidden or latent subspace. 

$z_t = [z_{t}^{obs}, z_{t}^{lat}]^{T}$

I modified this observation equation by adding a convolution with the $hrf$ function

$\hat{x}_{t} = B ( hrf \ast z_{t_0:t} )$

where the latent states are convolved over time with $hrf$ and then projected from the latent to the observation space

This includes the complication that the observationsat time $t$  $x_{t}$  don't just depend on the current state $z_{t}$, but on a set of states $z_{t_0:t}$ across serveral previous time steps.

## Modified Teacher Forcing

Given a PLRNN $F_{\theta}$, teacher forcing (TF) consists of regularly replacing the network's state variable $z_{t}$ with desired target values $d_t$. In weak teacher forcing, the generated state variable $z_t$ and the desired target $d_t$ are linearly interpolated using a weighting coefficient $\alpha$

$z_{t+1} = F_{\theta} (( 1-\alpha ) z_{t} + \alpha d_{t} )$ ,  $\alpha \in (0,1)$

The desired target is simply the ground truth for the observation subspace of the first $N$ dimensions and 0 in the hidden subspace of higher dimension $N+1$ to $M$

$ d_t = [x_{t}, z_{t}^{lat}]^{T} $

The problem with the convolutional observation model is that it is no longer clear what the forcing signal should look like because the observations $x_{t}$ depend on multiple  latant states.

I used the Wiener deconvolution on the ground truth $x$, which we assume is convolved with a $hrf$ function, to obtain an estimate $y$ of the unconvolved signal. This estimate was then passed as forcing signal

$ d_t = [y_{t}, z_{t}^{lat}]^{T} $

# Experimental Setup

### Benchmarking on data without noise

1. I created a dataset of 10^5 timesteps of the Lorenz63 attractor in the chaotic regime
2. I trained a PLRNN (shallowPLRNN) with 5000 epochs on the Lorenz63 dataset using weak Identity Teacher Forcing and Identity Mapping as Observation model
3. Using the model trained in 2. I created 4 datasets, each containing 100 trajectories drawn from the model. In the linear case I did not further modify the data. In the other cases I convoluted the data with a $hrf_{1.2}$, a $hrf_{0.5}$ and a $hrf_{0.2}$ respectively. These datasets served as the training data for the benchmark 
4. To benchmark, I trained 100 models with the linear Identity Mapping on each of the datasets. Then I trained 100 models with the convolutional Observation model on each dataset. For the convolution I used the same $hrf$ also used to convolve the data. In the case of the unconvolved dataset I used $hrf_{1.2}$. Each model was trained for 1000 epochs.
5. After every 25 training epochs a trajectory is generated by the model and the state space distance and Hellinger distance are calculated between the generated trajectory and the test set. In the Benchmark results I used the metrics from the epoch with the lowest state space distance.

![alt text](lorenz_hrf_conv.png "Title")

### Benchmarking on data with noise

1. I created a dataset of 10^5 timesteps of the Lorenz63 attractor in the chaotic regime
2. I trained a PLRNN (shallowPLRNN) with 5000 epochs on the Lorenz63 dataset using weak Identity Teacher Forcing and Identity Mapping as Observation model
3. Using the model trained in 2. I created 2 datasets, each containing 100 trajectories drawn from the model. I convoluted the data with a $hrf_{1.2}$ an added gaussian white noise with a mean $\mu=0$ and a variance of $\sigma=0.1$ and $\sigma=0.01$. These datasets served as the training data for the benchmark 
4. To benchmark, I trained 100 models with the linear Identity Mapping on both datasets. Then I trained 100 models with the convolutional Observation model on each dataset. For the convolution I used the same $hrf$ also used to convolve the data. Each model was trained for 1000 epochs.
5. After every 25 training epochs a trajectory is generated by the model and the state space distance and Hellinger distance are calculated between the generated trajectory and the test set. In the Benchmark results I used the metrics from the epoch with the lowest state space distance.

![alt text](lorenz_hrf_conv_noise.png "Title")